<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Characteristic GRMHD speeds
## Author: Patrick Nelson & Terrence Pierre Jacques

This notebook documents the function from the original `IllinoisGRMHD` code that calculates the speeds of the characteristic GRMHD waves.

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code produces the expected C code for the generated functions.

### NRPy+ Source Code for this module: 
* [IGM_Characteristic_Speeds.py](/edit/fluxes/IGM_Characteristic_Speeds.py).

## Introduction

Here, we will find the speeds at which the magnetohydrodynamic waves propagate. We find the speeds $c_+$ and $c_-$ on each cell interface with the function `find_cp_cm`; then, we find minimum and maximum speeds possible from among those. A complete derivation of these speeds is included [below](#derive_speed) as an appendix; here, for brevity, we simply start with the implementation of the code in the original `IllinoisGRMHD` code; we will implement simplifications to help improve performance. Below is the source code for `find_cp_cm`

```c
static inline void find_cp_cm(CCTK_REAL &cplus,CCTK_REAL &cminus,
                              CCTK_REAL v02,CCTK_REAL u0,
                              CCTK_REAL vi,CCTK_REAL ONE_OVER_LAPSE_SQUARED,
                              CCTK_REAL shifti,CCTK_REAL psim4,
                              CCTK_REAL gupii) {
    
  CCTK_REAL u0_SQUARED=SQR(u0);

  //Find cplus, cminus:
  CCTK_REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
  CCTK_REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
  CCTK_REAL c = u0_SQUARED*SQR(vi) * (1.0-v02) - v02 * ( psim4*gupii -
                                                         SQR(shifti)*ONE_OVER_LAPSE_SQUARED);
  CCTK_REAL detm = b*b - 4.0*a*c;
  //ORIGINAL LINE OF CODE:
  //if(detm < 0.0) detm = 0.0;
  //New line of code (without the if() statement) has the same effect:
  detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */

  cplus = 0.5*(detm-b)/a;
  cminus = -0.5*(detm+b)/a;
  if (cplus < cminus) {
    CCTK_REAL cp = cminus;
    cminus = cplus;
    cplus = cp;
  }
}
```
Comments documenting this have been excised for brevity, but are reproduced in $\LaTeX$ [below](#derive_speed).

One key feature in the C code generated here is the removal of `if` statements. These can slow down otherwise performant code, so we replace them with functions defined in the NRPy+ module `Min_Max_and_Piecewise_Expressions.py`, which uses absolute value functions to circumvent the conditional statements. We also use the module implement `MIN` and `MAX` functions, again using absolute value functions.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#char_speeds): Calculating Characteristic Speeds at Interfaces
    1. [Step 2.a](#cp_cm): Compute $c_+$ and $c_-$
    1. [Step 2.b](#cmax_cmin): Compute $c_\max$ and $c_\min$
1. [Step 3](#C_code): C Code Generation
    1. [Step 3.a](#C_code_a): Computing the Characteristic Speed Expressions
    1. [Step 3.b](#C_code_b): Accessing Structs
    1. [Step 3.c](#C_code_c): Printing the C Functions
1. [Step 4](#code_validation): Code Validation against `IGM_Characteristic_Speeds` Module
1. [Step 5](#derive_speed): Complete Derivation of the Wave Speeds
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file


<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code imports the core NRPy+ functionality after first adding the main NRPy+ directory to the path. 

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../../GRHayL/Flux_Source/")
sys.path.append(GRMHD_dir_path)

nrpy_dir_path = os.path.join("../../GRHayL/Flux_Source/nrpy/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

from outputC import outputC, outCfunction  # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "IGM-fluxes"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

<a id='find_cp_cm'></a>

# Step 2: Calculating Characteristic Speeds at Interfaces \[Back to [top](#toc)\]
$$\label{find_cp_cm}$$

We will now explain the inlined function `find_cp_cm`. Keep in mind that this function depends on the function `compute_v02`, [which is implemented below](#compute_v02). This function is called with the objective of computing the minimum ($-$) and maximum ($+$) characteristic speeds at each cell interface, $c_{\pm}^{r,l}$.

We approximate the general GRMHD dispersion relation (eq. 27 of [Gammie & McKinney (2003)](https://arxiv.org/pdf/astro-ph/0301509.pdf)) by the simpler expression

$$
\omega_{\rm cm}^{2} = \left[v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\right]k_{\rm cm}^{2}\ ,
$$

where $\omega_{\rm cm}=-k_{\mu}u^{\mu}$ is the frequency and $k_{\rm cm}^{2} = K_{\mu}K^{\mu}$ the wavenumber of an MHD wave mode in the frame comoving with the fluid, where $K_{\mu}$ is defined as the projection of the wave vector $k^{\nu}$ onto the direction normal to $u^{\nu}$: $K_{\mu} = \left(g_{\mu\nu}+u_{\mu}u_{\nu}\right)k^{\nu}$. $c_{\rm s}$ is the sound speed, and $v_{\rm A}$ is the Alfvén speed, given by

$$
v_{\rm A} = \sqrt{\frac{b^{2}}{\rho_{b}h + b^{2}}}\ .
$$


<a id='cp_cm'></a>

## Step 2.a: Compute $c_+$ and $c_-$ \[Back to [top](#toc)\]
$$\label{cp_cm}$$


With these definitions, we may then solve the approximate dispersion relation above along direction $i$, noting that in the comoving frame $k_{\mu} = \left(-\omega,k_{j}\delta^{j}_{\ i}\right)$ and the wave (phase) velocity is $c_{\pm} = \left.\omega\middle/\left(k_{j}\delta^{j}_{\ i}\right)\right.$. The dispersion can then be written as a quadratic equation for $c_{\pm}$:

$$
ac_{\pm}^{2} + bc_{\pm} + c = 0\ ,
$$

with

$$
\boxed{
\begin{align}
a &= \left(1-v_{0}^{2}\right)\left(u^{0}\right)^{2} - v_{0}^{2}g^{00}\ ,\\
b &= 2v_{0}^{2}g^{i0} - 2u^{i}u^{0}\left(1-v^{2}_{0}\right)\ ,\\
c &= \left(1-v_{0}^{2}\right)\left(u^{i}\right)^{2} - v_{0}^{2}g^{ii}\ ,\\
v_{0}^{2} &= v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right)\ ,\\
c_{\rm s}^2 &= \left.\left[\frac{dP_{\rm cold}}{d\rho_{b}} + \Gamma_{\rm th}\left(\Gamma_{\rm th}-1\right)\epsilon_{\rm th}\right]\middle/h\right.\ ,\\
c_{+} &= \max\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ ,\\
c_{-} &= \min\left(\frac{-b \pm \sqrt{b^{2}-4ac}}{2a}\right)\ .
\end{align}
}
$$

See eq. 50 and directly below that in [Duez, Liu, Shapiro, and Stephens](https://arxiv.org/pdf/astro-ph/0503420.pdf) for sound speed.

In [2]:
# We'll write this as a function so that we can calculate the expressions on-demand for any choice of i
def find_cp_cm(flux_dirn, g4UU, 
               smallbsquared, u4U, rho_b, h, cs2):
    # Outputs: cplus,cminus
    vA2 = smallbsquared / (rho_b*h + smallbsquared)
    v02 = vA2 + (cs2)*(1 - vA2)
    
    a = (1 - v02)*(u4U[0]**2) - v02*g4UU[0][0]
    b = 2*v02*g4UU[flux_dirn+1][0] - 2*u4U[flux_dirn+1]*u4U[0]*(1 - v02)
    c = (1 - v02)*(u4U[flux_dirn+1]**2) - v02*g4UU[flux_dirn+1][flux_dirn+1]

    # Now, we are free to solve the quadratic equation as usual. We take care to avoid passing a
    # negative value to the sqrt function.
    detm = b*b - sp.sympify(4)*a*c

    import Min_Max_and_Piecewise_Expressions as noif
    detm = sp.sqrt(noif.max_noif(sp.sympify(0),detm))
    global cplus,cminus
    # note that these correspond to a single interface, left or right
    cplus_tmp  = sp.Rational(1,2)* (-b/a + detm/a)
    cminus_tmp = sp.Rational(1,2)*-( b/a + detm/a)
    
    cminus = noif.min_noif(cplus_tmp, cminus_tmp)
    cplus  = noif.max_noif(cplus_tmp, cminus_tmp)

    # the above in C code
    # if (cplus < cminus) {
    # CCTK_REAL cp = cminus;
    # cminus = cplus;
    # cplus = cp;

<a id='cmax_cmin'></a>

## Step 2.b: Compute $c_\max$ and $c_\min$ \[Back to [top](#toc)\]
$$\label{cmax_cmin}$$

Using the function defined above, we now compute the maximum and mininum characteristic speeds. We note that our calculation is off by about a factor of 2, adding more diffusion to our overal scheme. See full discussion around Eqs. 49 and 50 in [Duez, Liu, Shapiro, and Stephens](https://arxiv.org/pdf/astro-ph/0503420.pdf).

In [3]:
# We'll write this as a function, and call it within HLLE_solver, below.
def find_cmax_cmin(flux_dirn, gamma_faceDD, beta_faceU, alpha_face,
                   smallbsquared_r, smallbsquared_l, 
                   u4U_r, u4U_l, rho_b_r, rho_b_l, 
                   h_r, h_l, cs2_r, cs2_l):
    # Inputs:  flux direction flux_dirn, Inverse metric g4_faceUU, shift beta_faceU,
    #          lapse alpha_face, metric determinant gammadet_face
    # Outputs: maximum and minimum characteristic speeds cmax and cmin
    # First, we need to find the characteristic speeds on each face
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4UU_ito_BSSN_or_ADM("ADM",gamma_faceDD, beta_faceU, alpha_face)

    # Original needed for GRMHD
    find_cp_cm(flux_dirn, AB4m.g4UU, smallbsquared_r, u4U_r, rho_b_r, h_r, cs2_r)
    cpr = cplus
    cmr = cminus
    
    find_cp_cm(flux_dirn, AB4m.g4UU, smallbsquared_l, u4U_l, rho_b_l, h_l, cs2_l)
    cpl = cplus
    cml = cminus

    # The following algorithms have been verified with random floats:

    global cmax,cmin    
#   // Then compute cmax, cmin. This is required for the HLL flux.
#   original C code
#   CCTK_REAL cmaxL =  MAX(0.0,MAX(cplusl,cplusr));
#   CCTK_REAL cminL = -MIN(0.0,MIN(cminusl,cminusr));
    
    # Now, we need to set cmax to the larger of cpr,cpl, and 0
    import Min_Max_and_Piecewise_Expressions as noif
    cmax = noif.max_noif(0.0, noif.max_noif(cpl, cpr))
    # And then, set cmin to the smaller of cmr,cml, and 0
    cmin = -noif.min_noif(0.0, noif.min_noif(cml, cmr))

<a id='C_code'></a>

# Step 3: C Code Generation \[Back to [top](#toc)\]
$$\label{C_code}$$

In this section we now generate the C code to do these calculations.


<a id='C_code_a'></a>

## Step 3.a: Computing the Characteristic Speed Expressions​ \[Back to [top](#toc)\]
$$\label{C_code_a}$$

Here we define the variables needed to calculate the speeds at the right and left interfaces. Furthermore, that because we are working in 3 dimension, we must calculate the speeds along each direction. Here we call on functions within [GRMHD_equations_new_version.py](/edit/GRMHD_formulation/GRMHD_equations_new_version.py) to aid in calculating the magnetic pressure.

In [4]:
formalism="ADM"

sqrt4pi = sp.symbols("SQRT_4_PI")
alpha_face = sp.symbols("alpha_face")
if formalism=="BSSN":
    cf_face = sp.symbols("cf_face")
    h_faceDD = ixp.declarerank2("h_faceDD","sym01",DIM=3)
    vet_faceU = ixp.declarerank1("vet_faceU", DIM=3)
    beta_faceU = vet_faceU
    
    import BSSN.ADM_in_terms_of_BSSN as AitoB
    AitoB.ADM_in_terms_of_BSSN()

    import BSSN.BSSN_quantities as Bq

    gamma_faceDD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            gamma_faceDD[i][j] = AitoB.gammaDD[i][j].subs(Bq.hDD[i][j], h_faceDD[i][j]).subs(Bq.cf, cf_face)

else:
    beta_faceU = ixp.declarerank1("beta_faceU") 
    gamma_faceDD = ixp.declarerank2("gamma_faceDD","sym01")
            
            
# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4rU = ixp.declarerank1("u4rU", DIM=4)
u4lU = ixp.declarerank1("u4lU", DIM=4)

BrU = ixp.declarerank1("BrU")
BlU = ixp.declarerank1("BlU")

rho_b_r = sp.symbols("rhob_r")
rho_b_l = sp.symbols("rhob_l")

h_r = sp.symbols("h_r")
h_l = sp.symbols("h_l")

cs2_r = sp.symbols("cs2_r")
cs2_l = sp.symbols("cs2_l")

GRMHD.compute_smallb4U(gamma_faceDD, beta_faceU, alpha_face, u4rU, BrU, sqrt4pi)
GRMHD.compute_smallbsquared(gamma_faceDD, beta_faceU, alpha_face, GRMHD.smallb4U)
smallbsquared_r = GRMHD.smallbsquared

GRMHD.compute_smallb4U(gamma_faceDD, beta_faceU, alpha_face, u4lU, BlU, sqrt4pi)
GRMHD.compute_smallbsquared(gamma_faceDD, beta_faceU, alpha_face, GRMHD.smallb4U)
smallbsquared_l = GRMHD.smallbsquared

In [5]:
# GRHayL structs:

# typedef struct primitive_quantities {
# double rho, press, eps;
# double vx, vy, vz;
# double Bx, By, Bz;
# double entropy, Y_e, temperature;
# double enthalpy;
# } primitive_quantities;

# typedef struct conservative_quantities {
#   double rho, tau, Y_e;
#   double S_x, S_y, S_z;
#   double entropy;
# } conservative_quantities;

# typedef struct metric_quantities {
#   double adm_gxx, adm_gxy, adm_gxz;
#   double adm_gyy, adm_gyz, adm_gzz;
#   double adm_gupxx, adm_gupxy, adm_gupxz;
#   double adm_gupyy, adm_gupyz, adm_gupzz;
#   double betax, betay, betaz;
#   double lapse, lapseinv;
#   double psi2, psi4, psi6;
#   double psi4inv, lapseinv2;
#   double g4dn[4][4],g4up[4][4];
# } metric_quantities;

# typedef struct metric_derivatives {
#   double lapse[3];
#   double betax[3];
#   double betay[3];
#   double betaz[3];
#   double adm_gxx[3];
#   double adm_gxy[3];
#   double adm_gxz[3];
#   double adm_gyy[3];
#   double adm_gyz[3];
#   double adm_gzz[3];
# } metric_derivatives;

In [6]:
cmins_rhs = []
cmaxs_rhs = []

for flux_dirn in range(3):
    find_cmax_cmin(flux_dirn, gamma_faceDD, beta_faceU, alpha_face,
                   smallbsquared_r, smallbsquared_l, 
                   u4rU, u4lU, rho_b_r, rho_b_l,
                   h_r, h_l, cs2_r, cs2_l,)
    
    cmins_rhs.append(cmin)
    cmaxs_rhs.append(cmax)

<a id='C_code_b'></a>

## Step 3.b: Accessing Structs \[Back to [top](#toc)\]
$$\label{C_code_b}$$

Because each gem within GRHayL is meant to be used in a point-wise fashion, we store values within structs and access them within a given C function. The below simply prints the C code necessary to access data within the relevant structs, which are __reconstructed_prims_r__, __reconstructed_prims_l__, and __metric_face_quantities__.

In [7]:
prims_NRPy_r = ["u4rU0", "u4rU1", "u4rU2", "u4rU3", "BrU0", "BrU1", "BrU2", "rhob_r"]
prims_NRPy_l = ["u4lU0", "u4lU1", "u4lU2", "u4lU3", "BlU0", "BlU1", "BlU2", "rhob_l"]

prims_GRHayL = ["u0", "vx", "vy", "vz", "Bx", "By", "Bz", "rho"]

prestring = r"""
double h_r, h_l, cs2_r, cs2_l;

eos->compute_h_and_cs2(eos, prims_r, &h_r, &cs2_r);
eos->compute_h_and_cs2(eos, prims_l, &h_l, &cs2_l);
"""

for i in range(len(prims_NRPy_r)):
    if "v" in prims_GRHayL[i]: 
        vel_r_str = prims_NRPy_r[0]
        vel_l_str = prims_NRPy_l[0]
        prestring += "const double "+prims_NRPy_r[i]+" = prims_r->"+prims_GRHayL[i]+"*"+vel_r_str+";\n"
        prestring += "const double "+prims_NRPy_l[i]+" = prims_l->"+prims_GRHayL[i]+"*"+vel_l_str+";\n"
    else:
        prestring += "const double "+prims_NRPy_r[i]+" = prims_r->"+prims_GRHayL[i]+";\n"
        prestring += "const double "+prims_NRPy_l[i]+" = prims_l->"+prims_GRHayL[i]+";\n"

prestring += "const double "+str(alpha_face)+" = metric_face->lapse;\n"

checker = []

if formalism=="BSSN":
    prestring += "const double "+str(cf_face)+" = metric_face_quantities->"+str(cf_face)+";\n"

    for i in range(3):
        vetU_var = vet_faceU[i]
        prestring += "const double "+str(vetU_var)+" = metric_face_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            hDD_var = h_faceDD[i][j]
            if hDD_var in checker:
                continue
            prestring += "const double "+str(hDD_var)+" = metric_face_quantities->"+str(hDD_var)+";\n"
            checker.append(hDD_var)

else:
    
    prestring += "const double beta_faceU0 = metric_face->betax;\n"
    prestring += "const double beta_faceU1 = metric_face->betay;\n"
    prestring += "const double beta_faceU2 = metric_face->betaz;\n"

    prestring += "const double gamma_faceDD00 = metric_face->adm_gxx;\n"
    prestring += "const double gamma_faceDD01 = metric_face->adm_gxy;\n"
    prestring += "const double gamma_faceDD02 = metric_face->adm_gxz;\n"

    prestring += "const double gamma_faceDD11 = metric_face->adm_gyy;\n"
    prestring += "const double gamma_faceDD12 = metric_face->adm_gyz;\n"

    prestring += "const double gamma_faceDD22 = metric_face->adm_gzz;\n"
    
#     for i in range(3):
#             betaU_var = beta_faceU[i]
#             prestring += "const double "+str(betaU_var)+" = metric_face_quantities->"+str(betaU_var)+";\n"

#     for i in range(3):
#         for j in range(3):
#             gammaDD_var = gamma_faceDD[i][j]
#             if gammaDD_var in checker:
#                 continue
#             prestring += "const double "+str(gammaDD_var)+" = metric_face_quantities->"+str(gammaDD_var)+";\n"
#             checker.append(gammaDD_var)

<a id='C_code_c'></a>

## Step 3.c: Printing the C Functions \[Back to [top](#toc)\]
$$\label{C_code_c}$$

Here we now print out the C functions that will be used in a point-wise fashion to calculate the characteristic speeds along each direction.

In [8]:
outCparams = "outCverbose=False,CSE_sorting=True"

cmins = ["cmin_dirn0", 
         "cmin_dirn1", 
         "cmin_dirn2"]

cmaxs = ["cmax_dirn0", 
         "cmax_dirn1", 
         "cmax_dirn2"]

c_type = "void"

params  =  "const primitive_quantities *restrict prims_r, "
params  += "const primitive_quantities *restrict prims_l, "
params  += "struct eos_parameters const *restrict eos, "
params  += "const metric_quantities *restrict metric_face, "


for flux_dirn in range(3):
    cmin_param = "double *"+cmins[flux_dirn]+", "
    cmax_param = "double *"+cmaxs[flux_dirn]

    write_speeds_str = ["*"+cmins[flux_dirn], "*"+cmaxs[flux_dirn]]
    write_speeds_rhs_str = [cmins_rhs[flux_dirn], cmaxs_rhs[flux_dirn]]

    body = outputC(write_speeds_rhs_str, write_speeds_str, params=outCparams, 
                   filename="returnstring", prestring=prestring)

    desc = "Compute the characteristic speeds in direction " + str(flux_dirn)
    name = "calculate_characteristic_speed_dirn" + str(flux_dirn)
    includes = ["NRPy_basic_defines.h"]

    outCfunction(
        outfile=os.path.join(Ccodesdir,name+".c"),
        includes=includes,
        desc=desc,
        name=name,
        params=params + cmin_param + cmax_param,
        body= body, 
        enableCparameters=False)

Output C function calculate_characteristic_speed_dirn0() to file IGM_standalone_Ccodes/calculate_characteristic_speed_dirn0.c
Output C function calculate_characteristic_speed_dirn1() to file IGM_standalone_Ccodes/calculate_characteristic_speed_dirn1.c
Output C function calculate_characteristic_speed_dirn2() to file IGM_standalone_Ccodes/calculate_characteristic_speed_dirn2.c


<a id='code_validation'></a>

# Step 4:  Code Validation against `IGM_Characteristic_Speeds` Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the C code generated by 
1. this tutorial and 
2. the [IGM_Characteristic_Speeds.py](/edit/fluxes/IGM_Characteristic_Speeds.py) module.


In [9]:
files = ["calculate_characteristic_speed_dirn0.c",
         "calculate_characteristic_speed_dirn1.c",
         "calculate_characteristic_speed_dirn2.c"]

In [10]:
# Define the directory that we wish to validate against:
valdir = "IGM_Ccode_library/"
cmd.mkdir(os.path.join(valdir))

import IGM_Characteristic_Speeds as chsp
chsp.Cfunction__GRMHD_characteristic_speeds(valdir, includes=includes,
                                                    formalism=formalism,
                                                    outCparams=outCparams)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .c file. See differences above.")
            sys.exit(1)

Output C function calculate_characteristic_speed_dirn0() to file IGM_Ccode_library/calculate_characteristic_speed_dirn0.c
Output C function calculate_characteristic_speed_dirn1() to file IGM_Ccode_library/calculate_characteristic_speed_dirn1.c
Output C function calculate_characteristic_speed_dirn2() to file IGM_Ccode_library/calculate_characteristic_speed_dirn2.c
Printing difference between original C code and this code...
Checking file calculate_characteristic_speed_dirn0.c
No difference. TEST PASSED!
Checking file calculate_characteristic_speed_dirn1.c
No difference. TEST PASSED!
Checking file calculate_characteristic_speed_dirn2.c
No difference. TEST PASSED!


<a id='derive_speed'></a>

# Step 5: Complete Derivation of the Wave Speeds \[Back to [top](#toc)\]
$$\label{derive_speed}$$

This computes phase speeds in the direction given by `flux_dirn`. Note that we replace the full dispersion relation with a simpler one, which overestimates the maximum speeds by a factor of ~2. See full discussion around Eqs. 49 and 50 in [Duez, et al.](http://arxiv.org/pdf/astro-ph/0503420.pdf). In summary, we solve the dispersion relation (in, e.g., the $x$-direction) with a wave vector of $k_\mu = (-\omega,k_x,0,0)$. So, we solve the approximate dispersion relation $\omega_{\rm cm}^2 = [v_A^2 + c_s^2 (1-v_A^2)]k_{\rm cm}^2$ for the wave speed $\omega/k_x$, where the sound speed $c_s = \sqrt{\Gamma P/(h \rho_0)}$, the Alfv&eacute;n speed $v_A = 1$ (in GRFFE), $\omega_{\rm cm} = -k_\mu k^\mu$ is the frequency in the comoving frame, $k_{\rm cm}^2 = K_\mu K^\mu$ is the wavenumber squared in the comoving frame, and $K_\mu = (g_{\mu\nu} + u_\mu u_\nu)k^\nu$ is the part of the wave vector normal to the four-velocity $u^\mu$. See below for a complete derivation.

What follows is a complete derivation of the quadratic we solve. We start from the following relations:
\begin{align}
w_{\rm cm} &= (-k_0 u^0 - k_x u^x) \\
k_{\rm cm}^2 &= K_{\mu} K^{\mu}, \\
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
\end{align}
The last term of the above can be written as follow:
$$
(g_{c b} + u_{c} u_{b}) k^c = (\delta^{\mu}_c + u_c u^{\mu} ) k^c
$$

Then,
\begin{align}
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
                 &= (g_{\mu a} + u_{\mu} u_a) k^a  (\delta^{\mu}_c + u_c u^{\mu} ) k^c \\
                 &=[(g_{\mu a} + u_{\mu} u_a) \delta^{\mu}_c + (g_{\mu a} + u_{\mu} u_a) u_c u^{\mu} ] k^c k^a \\
                 &=[(g_{c a} + u_c u_a) + (u_c u_a -  u_a u_c] k^c k^a \\
                 &=(g_{c a} + u_c u_a) k^c k^a \\
                 &= k_a k^a + u^c u^a k_c k_a \\
k^a = g^{\mu a} k_{\mu} &= g^{0 a} k_0 + g^{x a} k_x \\
k_a k^a &= k_0 g^{0 0} k_0 + k_x k_0 g^{0 x} + g^{x 0} k_0 k_x + g^{x x} k_x k_x \\
         &= g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2 \\
u^c u^a k_c k_a &= (u^0 k_0 + u^x k_x) (u^0 k_0 + u^x k_x) = (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 \\
(k_0 u^0)^2  + 2 k_x u^x k_0 u^0 + (k_x u^x)^2 &= v_0^2 [ (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 + g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2] \\
(1-v_0^2) (u^0 k_0 + u^x k_x)^2 &= v_0^2 (g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2) \\
(1-v_0^2) (u^0 k_0/k_x + u^x)^2 &= v_0^2 (g^{00} (k_0/k_x)^2 + 2 g^{x0} k_0/k_x + g^{xx}) \\
(1-v_0^2) (u^0 X + u^x)^2 &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
(1-v_0^2) ((u^0)^2 X^2 + 2 u^x (u^0) X + (u^x)^2) &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
0 &= X^2 ( (1-v_0^2) (u^0)^2 - v_0^2 g^{00}) + X (2 u^x u^0 (1-v_0^2) - 2 v_0^2 g^{x0}) + (1-v_0^2) (u^x)^2 - v_0^2 g^{xx} \\
a &= (1-v_0^2) (u^0)^2 - v_0^2 g^{00} = (1-v_0^2) (u^0)^2 + v_0^2/\alpha^2 \leftarrow {\rm VERIFIED} \\
b &= 2 u^x u^0 (1-v_0^2) - 2 v_0^2 \beta^x/\alpha^2 \leftarrow {\rm VERIFIED,\ } X\rightarrow -X, {\rm because\ } X = -w/k_1, {\rm \ and\ we\ are\ solving\ for} -X. \\
c &= (1-v_0^2) (u^x)^2 - v_0^2 (\gamma^{xx}\psi^{-4} - (\beta^x/\alpha)^2) \leftarrow {\rm VERIFIED} \\
v_0^2 &= v_A^2 + c_s^2 (1 - v_A^2) \\
\end{align}

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GRHayL-Characteristic_Speeds-NRPy_Gen.pdf](Tutorial-GRHayL-Characteristic_Speeds-NRPy_Gen.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GRHayL-Characteristic_Speeds-NRPy_Gen")

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/traitlets.py", line 537, in get
    value = obj._trait_values[self.name]
KeyError: 'template_paths'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/nbconvert/nbconvertapp.py", line 369, in start
    self.convert_notebooks()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packag